In [1]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.10.1


In [2]:
# Get the current working directory
# cwd = os.getcwd()

X, y = [], []
path = "wheat_leaf"
class_dict = {'Healthy':0, 
              'septoria':1, 
              'stripe_rust':2}

for each_class in class_dict.keys():
    # Get all the files in the path directory
    files = os.listdir(path + '/' + each_class)
    # print(files)
    for each_file in files:
        if each_file[0] == '.':
            continue
            
#         image = tf.keras.utils.load_img(path + '/' + each_class + '/' + each_file, 
#                                         target_size=(224, 224))
#         image_arr = tf.keras.utils.img_to_array(image)
        img = cv.imread(path + '/' + each_class + '/' + each_file)
        image_arr = cv.resize(img, (224,224))
        X.append(image_arr)
        y.append(class_dict[each_class])
    print('successfully loaded for class:',  each_class)

successfully loaded for class: Healthy
successfully loaded for class: septoria
successfully loaded for class: stripe_rust


In [3]:
X, y = np.array(X), np.array(y)
X.shape, y.shape

((407, 224, 224, 3), (407,))

In [4]:
from sklearn.model_selection import train_test_split

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

X_train, X_dummy, y_train, y_dummy = train_test_split(X, y, train_size=0.9, shuffle=True, random_state=123, stratify=y)
print(f'training',X_train.shape, X_dummy.shape, y_train.shape, y_dummy.shape)

X_val, X_test, y_val, y_test = train_test_split(X_dummy, y_dummy, train_size=0.5, shuffle=True, random_state=123, stratify=y_dummy)
print(f'val - test',X_val.shape, X_test.shape, y_val.shape, y_test.shape)


training (366, 224, 224, 3) (41, 224, 224, 3) (366,) (41,)
val - test (20, 224, 224, 3) (21, 224, 224, 3) (20,) (21,)


In [5]:
X_train = X_train / 255
X_test = X_test / 255
X_val = X_val / 255

In [6]:
# y_train.value_counts()
unique_values, counts = np.unique(y_train, return_counts=True)
unique_values, counts, (counts / len(y_train) * 100).round(2)

(array([0, 1, 2]),
 array([ 92,  87, 187], dtype=int64),
 array([25.14, 23.77, 51.09]))

In [7]:
# y_test.value_counts()
unique_values, counts = np.unique(y_test, return_counts=True)
unique_values, counts, (counts / len(y_test) * 100).round(2)

(array([0, 1, 2]),
 array([ 5,  5, 11], dtype=int64),
 array([23.81, 23.81, 52.38]))

In [8]:
# y_val.value_counts()
unique_values, counts = np.unique(y_val, return_counts=True)
unique_values, counts, (counts / len(y_test) * 100).round(2)

(array([0, 1, 2]),
 array([ 5,  5, 10], dtype=int64),
 array([23.81, 23.81, 47.62]))

In [9]:
# Convert labels to one-hot encoding
num_classes = 3
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_train.shape, y_test.shape, y_val.shape

((366, 3), (21, 3), (20, 3))

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [11]:
# Define input image dimensions
img_width, img_height = 224, 224
num_classes = 3  # Example number of classes

In [12]:
# Load the MobileNetV2 model pre-trained on ImageNet data
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the convolutional base
base_model.trainable = False

In [13]:
# Add custom dense layers on top of the convolutional base
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [14]:
# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

In [15]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [16]:
# Define data augmentation parameters
train_datagen = ImageDataGenerator(
    rotation_range=20,      # Random rotation within the range [-20, 20] degrees
    width_shift_range=0.1,  # Random horizontal shift by up to 10% of the image width
    height_shift_range=0.1, # Random vertical shift by up to 10% of the image height
    zoom_range=0.2,         # Random zoom by up to 20%
    horizontal_flip=True,   # Random horizontal flipping
    fill_mode='nearest'
)

In [17]:
# Train the model
history = model.fit(train_datagen.flow(X_train, y_train, batch_size=64),
                    epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
6/6 [==============================] - 10s 1s/step - loss: 0.9762 - accuracy: 0.6011 - val_loss: 0.3614 - val_accuracy: 0.8500
Epoch 2/10
6/6 [==============================] - 7s 1s/step - loss: 0.3320 - accuracy: 0.8689 - val_loss: 0.1158 - val_accuracy: 0.9500
Epoch 3/10
6/6 [==============================] - 7s 1s/step - loss: 0.2182 - accuracy: 0.9208 - val_loss: 0.0709 - val_accuracy: 1.0000
Epoch 4/10
6/6 [==============================] - 7s 1s/step - loss: 0.1718 - accuracy: 0.9399 - val_loss: 0.0639 - val_accuracy: 1.0000
Epoch 5/10
6/6 [==============================] - 8s 1s/step - loss: 0.1193 - accuracy: 0.9590 - val_loss: 0.0832 - val_accuracy: 0.9500
Epoch 6/10
6/6 [==============================] - 6s 975ms/step - loss: 0.0899 - accuracy: 0.9727 - val_loss: 0.0566 - val_accuracy: 1.0000
Epoch 7/10
6/6 [==============================] - 8s 1s/step - loss: 0.0816 - accuracy: 0.9754 - val_loss: 0.0523 - val_accuracy: 1.0000
Epoch 8/10
6/6 [=====================

In [19]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

1/1 [==============================] - 0s 237ms/step - loss: 0.1574 - accuracy: 0.9524
Test loss: 0.1573726385831833
Test accuracy: 0.9523809552192688
